In [1]:
# Import dependencies
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import re

In [2]:
# Set Chromedriver path
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# News Title and Paragraph Text

In [3]:
# Visit the following URL
url_news = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url_news)

In [4]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

news_title = soup.find('div', class_='content_title').text
news_para = soup.find('div', class_= 'article_teaser_body').text

In [5]:
news_para

"One month since increasing their commanding frequency, engineers have yet to hear from NASA's Opportunity rover."

# Find JPL Image

In [6]:
# Visit the following URL
url_jpl = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_jpl)

In [7]:
# Design an XPATH selector to grab the "JPL featured image"
xpath = '//footer//a[@class="button fancybox"]'

In [8]:
# to bring up the full resolution image
results = browser.find_by_xpath(xpath)
img = results[0]
img.click()

In [9]:
# Scrape the browser into soup and use soup to find the full resolution image of mars
browser.is_element_present_by_css("img.jpg", wait_time=1)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
featured_img_url = soup.find("img", class_="fancybox-image")["src"]
if "https:" not in featured_img_url: featured_img_url = "https://www.jpl.nasa.gov"+featured_img_url
featured_img_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19643_ip.jpg'

In [10]:
# Use the requests library to download and save the image from the `img_url` above
import requests
import shutil
response = requests.get(featured_img_url, stream=True)
with open('img.png', 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)

In [11]:
# Display the image with IPython.display
from IPython.display import Image
Image(url='img.png')

# Mars Weather

In [12]:
# Visit the following URL
url_weather = "https://twitter.com/marswxreport?lang=en"
browser.visit(url_weather)

In [13]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [14]:
mars_weather = soup.find(string=re.compile("Sol"))
print(mars_weather) 

Sol 2171 (2018-09-14), high -12C/10F, low -65C/-84F, pressure at 8.79 hPa, daylight 05:43-17:59


# Mars Facts

In [15]:
# Visit the following URL
url_fact = "https://space-facts.com/mars/"
browser.visit(url_fact)

In [16]:
table = pd.read_html(url_fact)
table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [17]:
df = table[0]
df.columns = ['Attribute', 'Unit']
df.head()

,Attribute,Unit
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"


In [18]:
df.set_index('Attribute', inplace=True)
df.head()

,Unit
Attribute,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"


In [19]:
html_table = df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Unit</th>\n    </tr>\n    <tr>\n      <th>Attribute</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n 

In [20]:
from pprint import pprint
pprint(html_table.replace('\n', ''))

('<table border="1" class="dataframe">  <thead>    <tr style="text-align: '
 'right;">      <th></th>      <th>Unit</th>    </tr>    <tr>      '
 '<th>Attribute</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      '
 '<th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      '
 '<th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      '
 '<th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    '
 '<tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    '
 '<tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.52 AU)</td>    '
 '</tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 '
 'years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      '
 '<td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record:</th>      '
 '<td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      '
 '<td>Egyptian astronomers</td>    </tr>  </tbody></table>')


# Mars Hemispheres

In [21]:
# Visit the following URL
url_hemi = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url_hemi)

In [22]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [23]:
hemisphere_image_urls = []
results = soup.find_all('h3')

In [24]:
xpath = '//a[@class="itemLink product-item"]/img'

In [25]:
for result in results:
    title = result.text
    title = title.replace(" Enhanced", "")
    
    browser.find_by_xpath(xpath)[results.index(result)].click()
    
    browser.is_element_present_by_css("img.jpg", wait_time=1)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    hemi_img_url = soup.find("img", class_="wide-image")["src"]
    if "https:" not in hemi_img_url:
        hemi_img_url = "https://astrogeology.usgs.gov"+hemi_img_url
    
    hemisphere_image_urls.append({'title': title,
                                 'img_url': hemi_img_url})
    
    browser.back()
    
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]

In [26]:
html_dict = {'news_title': news_title,
            'news_para': news_para,
            'featured_img_url': featured_img_url,
            'mars_weather': mars_weather,
            'html_table': html_table,
            'hemisphere_image_urls': hemisphere_image_urls}
html_dict

{'news_title': 'Update on Opportunity Rover after Martian Dust Storm',
 'news_para': "One month since increasing their commanding frequency, engineers have yet to hear from NASA's Opportunity rover.",
 'featured_img_url': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19643_ip.jpg',
 'mars_weather': 'Sol 2171 (2018-09-14), high -12C/10F, low -65C/-84F, pressure at 8.79 hPa, daylight 05:43-17:59',
 'html_table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Unit</th>\n    </tr>\n    <tr>\n      <th>Attribute</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      